#### Model Problem Setup

This notebook aims to implement the mathematical model from the reference paper [1] into Python code. Personnel competencies will be read from an Excel file that tabulates each individual's competenciesi taken directly from the paper. Please ensure you have installed ``IBM CPLEX``, ``docplex``, and ``pandas``. The code is made as simple as possible to illustrate the implementation of the mathematical model.

In [1]:
import pandas as pd
from docplex.mp.model import Model
from src.scheduller import Scheduler
from src.writer import Writer
from src.utils import list_range, competency_dict, save_solution_dict

competency_file = "./src/data/personnel_competencies.xlsx"
data_frame = pd.read_excel(competency_file)
competency = competency_dict(data_frame)

def person_competency(person_id: int, section_id: int):
    return competency.get(person_id).get(section_id)

# Create docplex Model
model = Model(name="ShiftScheduler")

##### Parameters
$$
\begin{aligned}
n & : \text{ number of personnel working in the factory, } n = 80 \\
m & : \text{ number of days, } m = 30 \\
s & : \text{ number of sections in the factory, } s = 7 \\
t & : \text{ number of shifts, } t = 2 \\
i & : \text{ personnel index, } i = 1, 2, \dots, n \\
j & : \text{ day index, } j = 1, 2, \dots, m \\
k & : \text{ section index, } k = 1, 2, \dots, s \\
l & : \text{ shift index, } l = 1, 2, \dots, t
\end{aligned}
$$


In [2]:
# Basic params
n = 80 	#num_personnel
m = 30 	#num_days, 
s = 7 	#num_sections, 
t = 2	#num_shifts

#### Decision Variables

Shift Decision Variables (X)

$$
X_{ijkl} = 
\begin{cases} 
1, & \text{if shift l, section k, and day j are chosen for personnel i} \\
0, & \text{otherwise}
\end{cases}, \quad i = 1, 2, \dots, n, \; j = 1, 2, \dots, m, \; k = 1, 2, \dots, s, \; l = 1, 2, \dots, t \quad\quad \text{(4)}
$$

In [3]:
X = {(i, j, k, l): model.binary_var(name=f"X_{i}_{j}_{k}_{l}")
    for i in list_range(n)
    for j in list_range(m)
    for k in list_range(s)
    for l in list_range(t)
}

Leave Desision Variables (h)
$$
h_{ij} = 
\begin{cases} 
1, & \text{if vacation for personnel \(i\) on shift \(j\)} \\
0, & \text{otherwise}
\end{cases}, \quad i = 1, 2, \dots, n, \; j = 1, 2, \dots, m  \quad\quad \text{(5)}
$$


In [4]:
h = {(i, j): model.binary_var(name=f"h_{i}_{j}") for i in list_range(n) for j in list_range(m)}

##### Deviation Variables

Deviation Goal 1

$$
d^-_{1ij}  \quad\quad  d^+_{1ij}
$$

In [5]:
d_neg_1 = model.continuous_var_matrix(keys1=range(1, n+1), keys2=range(1, m+1), name="d_neg_1")
d_pos_1 = model.continuous_var_matrix(keys1=range(1, n+1), keys2=range(1, m+1), name="d_pos_1")

Deviation Goal 2
$$
d^-_{2ij}  \quad\quad  d^+_{2ij}
$$

In [6]:
d_neg_2 = model.continuous_var_matrix(keys1=range(1, n+1), keys2=range(1, m+1), name="d_neg_2")
d_pos_2 = model.continuous_var_matrix(keys1=range(1, n+1), keys2=range(1, m+1), name="d_pos_2")

Deviation Goal 3
$$
d^-_{3ij}  \quad\quad  d^+_{3ij}
$$

In [7]:
d_neg_3 = model.continuous_var_list(keys=range(1, n + 1), name="d_neg_3")
d_pos_3 = model.continuous_var_list(keys=range(1, n + 1), name="d_pos_3")

Deviation Goal 4
$$
d^-_{4ij}  \quad d^+_{4ij} \quad d^-_{5ij}  \quad d^+_{5ij} \quad d^-_{6ij}  \quad d^+_{6ij} \quad d^-_{7ij}  \quad d^+_{7ij} \quad d^-_{8ij}  \quad d^+_{8ij} \quad d^-_{9ij}  \quad d^+_{9ij} \quad d^-_{10ij}  \quad d^+_{10ij} \quad 
$$

In [8]:
d_neg_4 = model.continuous_var_matrix(keys1=range(1, m+1), keys2=range(1, t+1), name="d_neg_4")
d_pos_4 = model.continuous_var_matrix(keys1=range(1, m+1), keys2=range(1, t+1), name="d_pos_4")
d_neg_5 = model.continuous_var_matrix(keys1=range(1, m+1), keys2=range(1, t+1), name="d_neg_5")
d_pos_5 = model.continuous_var_matrix(keys1=range(1, m+1), keys2=range(1, t+1), name="d_pos_5")
d_neg_6 = model.continuous_var_matrix(keys1=range(1, m+1), keys2=range(t+1), name="d_neg_6")
d_pos_6 = model.continuous_var_matrix(keys1=range(1, m+1), keys2=range(1, t+1), name="d_pos_6")
d_neg_7 = model.continuous_var_matrix(keys1=range(1, m+1), keys2=range(1, t+1), name="d_neg_7")
d_pos_7 = model.continuous_var_matrix(keys1=range(1, m+1), keys2=range(1, t+1), name="d_pos_7")
d_neg_8 = model.continuous_var_matrix(keys1=range(1, m+1), keys2=range(1, t+1), name="d_neg_8")
d_pos_8 = model.continuous_var_matrix(keys1=range(1, m+1), keys2=range(1, t+1), name="d_pos_8")
d_neg_9 = model.continuous_var_matrix(keys1=range(1, m+1), keys2=range(1, t+1), name="d_neg_9")
d_pos_9 = model.continuous_var_matrix(keys1=range(1, m+1), keys2=range(1, t+1), name="d_pos_9")
d_neg_10 = model.continuous_var_matrix(keys1=range(1, m+1), keys2=range(1, t+1), name="d_neg_10")
d_pos_10 = model.continuous_var_matrix(keys1=range(1, m+1), keys2=range(1, t+1), name="d_pos_10")

#### Objective Function
$$
\text{Min } Z = \sum_{i=1}^n \sum_{j=1}^m \left( d_{1ij}^- + d_{1ij}^+ \right) 
+ \left( d_{2ij}^- + d_{2ij}^+ \right) 
+ \left( d_{3i}^- + d_{3i}^+ \right) 
+ \sum_{j=1}^m \sum_{l=1}^t d_{4jl}^+ 
+ d_{5jl}^+ + d_{6jl}^+ + d_{7jl}^+ + d_{8jl}^+ + d_{9jl}^+ + d_{10jl}^ +
\quad \text(22)
$$

$$
\text{Personnel Objective } = \sum_{i=1}^n \sum_{j=1}^m \left( d_{1ij}^- + d_{1ij}^+ \right) 
+ \left( d_{2ij}^- + d_{2ij}^+ \right) 
+ \left( d_{3i}^- + d_{3i}^+ \right)
$$

$$
\text{Shift Objective } = \sum_{j=1}^m \sum_{l=1}^t d_{4jl}^+ 
+ d_{5jl}^+ + d_{6jl}^+ + d_{7jl}^+ + d_{8jl}^+ + d_{9jl}^+ + d_{10jl}^+
$$

In [9]:
model.minimize(
    # Goal 1 deviations
    model.sum(d_neg_1[i, j] + d_pos_1[i, j] for i in list_range(n) for j in list_range(m)) +
    # Goal 2 deviations
    model.sum(d_neg_2[i, j] + d_pos_2[i, j] for i in list_range(n) for j in list_range(m)) +
    # Goal 3 deviations
    model.sum(d_neg_3[i] + d_pos_3[i] for i in range(0, n)) +
    # Goals 4  deviations
    model.sum(d_pos_4[j, l] for j in list_range(m) for l in list_range(t)) +
    model.sum(d_pos_5[j, l] for j in list_range(m) for l in list_range(t)) +
    model.sum(d_pos_6[j, l] for j in list_range(m) for l in list_range(t)) +
    model.sum(d_pos_7[j, l] for j in list_range(m) for l in list_range(t)) +
    model.sum(d_pos_8[j, l] for j in list_range(m) for l in list_range(t)) +
    model.sum(d_pos_9[j, l] for j in list_range(m) for l in list_range(t)) +
    model.sum(d_pos_10[j, l] for j in list_range(m) for l in list_range(t))
)


#### Constraints

##### 1-To meet the daily personnel needs of the departments:

Number of personnel needed for each shift in the cutting(1) section.
$$
\sum_{i=1}^n \left( X_{ij1l} \right) = 3, \; j = 1, 2, \ldots, m, \; l = 1, 2  \quad\quad \text{(6)}
$$

In [10]:
for j in list_range(m):
    for l in list_range(t):
        model.add_constraint(
            model.sum(X[i, j, 1, l] for i in list_range(n)) == 3,
            f"Equation_6_Personnel_Requirement_Section_1_Day_{j}_Shift_{l}"
        )

Number of personnel needed for each shift in the sanding(2) section.
$$
\sum_{i=1}^n \left( X_{ij2l} \right) = 4, \; j = 1, 2, \ldots, m, \; l = 1, 2  \quad\quad \text{(7)}
$$

In [11]:
for j in list_range(m):
    for l in list_range(t):
        model.add_constraint(
            model.sum(X[i, j, 2, l] for i in list_range(n)) == 4,
            f"Equation_7_Personnel_Requirement_Section_2_Day_{j}_Shift_{l}"
        )

Number of personnel needed for each shift in the grinding(3) section.
$$
\sum_{i=1}^n \left( X_{ij3l} \right) = 4, \; j = 1, 2, \ldots, m, \; l = 1, 2  \quad\quad \text{(8)}
$$

In [12]:
for j in list_range(m):
    for l in list_range(t):
        model.add_constraint(
            model.sum(X[i, j, 3, l] for i in list_range(n)) == 4,
            f"Equation_8_Personnel_Requirement_Section_3_Day_{j}_Shift_{l}"
        )

Number of personnel needed for each shift in the tempering(4) section.
$$
\sum_{i=1}^n \left( X_{ij4l} \right) = 4, \; j = 1, 2, \ldots, m, \; l = 1, 2  \quad\quad \text{(9)}
$$

In [13]:
for j in list_range(m):
    for l in list_range(t):
        model.add_constraint(
            model.sum(X[i, j, 3, l] for i in list_range(n)) == 4,
            f"Equation_9_Personnel_Requirement_Section_4_Day_{j}_Shift_{l}"
        )

Number of personnel needed for each shift in the laminating(5) section.
$$
\sum_{i=1}^n \left( X_{ij5l} \right) = 6, \; j = 1, 2, \ldots, m, \; l = 1, 2  \quad\quad \text{(10)}
$$

In [14]:
for j in list_range(m):
    for l in list_range(t):
        model.add_constraint(
            model.sum(X[i, j, 5, l] for i in list_range(n)) == 6,
            f"Equation_10_Personnel_Requirement_Section_5_Day_{j}_Shift_{l}"
        )

Number of personnel needed for each shift in the glazing(6) section.
$$
\sum_{i=1}^n \left( X_{ij6l} \right) = 8, \; j = 1, 2, \ldots, m, \; l = 1, 2  \quad\quad \text{(11)}
$$

In [15]:
for j in list_range(m):
    for l in list_range(t):
        model.add_constraint(
            model.sum(X[i, j, 6, l] for i in list_range(n)) == 8,
            f"Equation_11_Personnel_Requirement_Section_6_Day_{j}_Shift_{l}"
        )

Number of personnel needed for each shift in the shipment(7) section.
$$
\sum_{i=1}^n \left( X_{ij7l} \right) = 5, \; j = 1, 2, \ldots, m, \; l = 1, 2  \quad\quad \text{(12)}
$$

In [16]:
for j in list_range(m):
    for l in list_range(t):
        model.add_constraint(
            model.sum(X[i, j, 7, l] for i in list_range(n)) == 5,
            f"Equation_12_Personnel_Requirement_Section_7_Day_{j}_Shift_{l}"
        )

##### 2- Only one shift per personnel per day:
$$
\sum_{l=1}^t \sum_{k=1}^s \left( X_{ijkl} \right) \leq 1, \; i = 1, 2, \ldots, n, \; j = 1, 2, \ldots, m  \quad\quad \text{(13)}
$$


In [17]:
for i in list_range(n):
    for j in list_range(m):
        model.add_constraint(
            model.sum(X[i, j, k, l] for k in list_range(s) for l in list_range(t)) <= 1,
            f"Equation_13_Single_Shift_Per_Day_{i}_{j}"
        )

##### 3- personnel not working on the day of leave:
$$
\sum_{l=1}^t \sum_{k=1}^s \left( X_{ijkl} \right) \leq \left( 1 - h_{ij} \right), \; i = 1, 2, \ldots, n, \; j = 1, 2, \ldots, m  \quad\quad\ \text{(14)}
$$


In [18]:
for i in list_range(n):
    for j in list_range(m):
        model.add_constraint(
            model.sum(X[i, j, k, l] for k in list_range(s) for l in list_range(t)) <= (1 - h[i, j]),
            f"Equation_14_No_Work_On_Vacation_{i}_{j}"
        )

#### 4- Each personnel member has a minimum of 1 and a maximum of 2 days a week.:

$$
h_{ij} + h_{i(j+1)} + h_{i(j+2)} + h_{i(j+3)} + h_{i(j+4)} + h_{i(j+5)} + h_{i(j+6)} \leq 2, \; i = 1, 2, \ldots, n, \; j = 1, 2, \ldots, m-6  \quad\quad \text{(15)}
$$

$$
h_{ij} + h_{i(j+1)} + h_{i(j+2)} + h_{i(j+3)} + h_{i(j+4)} + h_{i(j+5)} + h_{i(j+6)} \geq 1, \; i = 1, 2, \ldots, n, \; j = 1, 2, \ldots, m-6  \quad\quad \text{(16)}
$$


In [19]:

for i in list_range(n):
    for j in list_range(m - 6):
        # Equation 15: Maximum 2 vacations in a 7-day period
        model.add_constraint(
            model.sum(h[i, j + d] for d in range(7)) <= 2,
            f"Equation_15_Max_2_Vacations_7_Days_{i}_{j}"
        )
        # Equation 16: Minimum 1 vacation in a 7-day period
        model.add_constraint(
            model.sum(h[i, j + d] for d in range(7)) >= 1,
            f"Equation_16_Min_1_Vacation_7_Days_{i}_{j}"
        )

#### 5-Upper limit for each personnel to work on 1 and 2 shifts:
$$
\sum_{j=1}^m \sum_{k=1}^s \left( X_{ijk1} \right) \leq 12, \; i = 1, 2, \ldots, n \quad\quad \text{(17)}
$$

$$
\sum_{j=1}^m \sum_{k=1}^s \left( X_{ijk2} \right) \leq 12, \; i = 1, 2, \ldots, n \quad\quad \text{(18)}
$$


In [20]:
for i in list_range(n):
    # Equation 17: Total assignments to shift 1 across all days and sections
    model.add_constraint(
        model.sum(X[i, j, k, 1] for j in list_range(m) for k in list_range(s)) <= 12,
        f"Equation_17_Max_Shift_1_Assignments_{i}"
    )
    # Equation 18: Total assignments to shift 2 across all days and sections
    model.add_constraint(
        model.sum(X[i, j, k, 2] for j in list_range(m) for k in list_range(s)) <= 12,
        f"Equation_18_Max_Shift_2_Assignments_{i}"
    )

#### 6- Lower limit restrictions for each personnel on 1 and 2 shifts:
$$
\sum_{j=1}^m \sum_{k=1}^s \left( X_{ijk1} \right) \geq 10, \; i = 1, 2, \ldots, n \quad\quad \text{(19)}
$$

$$
\sum_{j=1}^m \sum_{k=1}^s \left( X_{ijk2} \right) \geq 10, \; i = 1, 2, \ldots, n \quad\quad \text{(20)}
$$


In [21]:
for i in list_range(n):
    # Equation 19: Minimum 10 assignments to shift 1 across all days and sections
    model.add_constraint(
        model.sum(X[i, j, k, 1] for j in list_range(m) for k in list_range(s)) >= 10,
        f"Equation_19_Min_Shift_1_Assignments_{i}"
    )
    # Equation 20: Minimum 10 assignments to shift 2 across all days and sections
    model.add_constraint(
        model.sum(X[i, j, k, 2] for j in list_range(m) for k in list_range(s)) >= 10,
        f"Equation_20_Min_Shift_2_Assignments_{i}"
    )

##### 7- If an employee were assigned to the night shift on a given day, the next day’s shift in the morning shift would be limited:

$$
\sum_{k=1}^s \left( X_{ijk2} \right) + \left( X_{i(j+1)k1} \right) \leq 1, \; i = 1, 2, \ldots, n, \; j = 1, 2 \ldots, m-1  \quad\quad \text{(21)}
$$


In [22]:
for i in list_range(n):
    for j in list_range(m - 1):
        model.add_constraint(
            model.sum(X[i, j, k, 2] for k in list_range(s)) + model.sum(X[i, j + 1, k, 1] for k in list_range(s)) <= 1,
            f"Equation_21_No_Night_To_Morning_Consecutive_Shift_{i}_{j}"
        )

## Goal Constraints

##### Goal 1: Goal constraint where personnel are asked to minimize the assignment as day of leave-workday-leave when being assigned shifts:
$$
h_{ij} + \sum_{k=1}^s \sum_{l=1}^t \left( X_{i(j+1)kl} \right) + h_{i(j+2)} + d_{1ij}^- - d_{1ij}^+ = 2, \; i = 1, 2, \ldots, n, \; j = 1, 2, \ldots, m-2 \quad\quad \text{(22)}
$$


In [23]:
for i in list_range(n):
    for j in list_range(m - 2):
        model.add_constraint(
            h[i, j] +
            model.sum(X[i, j + 1, k, l] for k in list_range(s) for l in list_range(t)) +
            h[i, j + 2] +
            d_neg_1[i, j] - d_pos_1[i, j] == 2,
            f"Goal_Constraint_Equation_22_{i}_{j}"
        )

##### Goal 2: Goal constraint where personnel are asked to minimize the assignment of working day-tracking-working day when being assigned to shifts:

$$
\sum_{k=1}^s \sum_{l=1}^t \left( X_{ijkl} \right) + h_{i(j+1)} + \sum_{k=1}^s \sum_{l=1}^t \left( X_{i(j+1)kl} \right) + d_{2ij}^- - d_{2ij}^+ = 2, \; i = 1, 2, \ldots, n, \; j = 1, 2, \ldots, m-2 \quad\quad \text{(23)}
$$


In [24]:
for i in list_range(n):
    for j in list_range(m - 2):
        model.add_constraint(
            model.sum(X[i, j, k, l] for k in list_range(s) for l in list_range(t)) +
            h[i, j + 1] +
            model.sum(X[i, j + 1, k, l] for k in list_range(s) for l in list_range(t)) +
            d_neg_2[i, j] - d_pos_2[i, j] == 2,
            f"Goal_Constraint_Equation_23_{i}_{j}"
        )

##### Goal 3: Goal constraint on which the total number of vacancies for which each personnel is assigned is intended to be as equal as possible:
$$
\sum_{j=1}^m \sum_{k=1}^s \sum_{l=1}^t \left( X_{ijkl} \right) + d_{3i}^- - d_{3i}^+ = 22, \; i = 1, 2, \ldots, n \quad\quad \text{(24)}
$$

In [25]:
for i in list_range(n):
    model.add_constraint(
        model.sum(X[i, j, k, l] for j in list_range(m) for k in list_range(s) for l in list_range(t)) +
        d_neg_3[i-1] - d_pos_3[i-1] == 22,
        f"Goal_Constraint_Equation_24_{i}"
    )

##### Goal 4: Personnel assigned to the departments in each shift will provide the required sum of points as a qualification:

$$
\sum_{i=1}^n \left( X_{ij1l} \right) * (1) + d_{4jl}^- - d_{4jl}^+ = 3, \; j = 1, 2, \ldots, m, \; l = 1, 2  \quad\quad \text{(25)}
$$



In [26]:
for j in list_range(m):
    for l in list_range(t):
        model.add_constraint(
            model.sum(X[i, j, 1, l] * person_competency(i, 1) for i in list_range(n)) + d_neg_4[j, l] - d_pos_4[j, l] == 3,
            f"Equation_25_{j}_{l}"
        )

$$
\sum_{i=1}^n \left( X_{ij2l} \right) * (2) + d_{5jl}^- - d_{5jl}^+ = 4, \; j = 1, 2, \ldots, m, \; l = 1, 2  \quad\quad \text{(26)}
$$

In [27]:
for j in list_range(m):
    for l in list_range(t):
        model.add_constraint(
            model.sum(X[i, j, 2, l] * person_competency(i, 2) for i in list_range(n)) + d_neg_5[j, l] - d_pos_5[j, l] == 4,
            f"Equation_26_{j}_{l}"
        )

$$
\sum_{i=1}^n \left( X_{ij3l} \right) * (3) + d_{6jl}^- - d_{6jl}^+ = 4, \; j = 1, 2, \ldots, m, \; l = 1, 2 \quad\quad \text{(27)}
$$

In [28]:
for j in list_range(m):
    for l in list_range(t):
        model.add_constraint(
            model.sum(X[i, j, 3, l] * person_competency(i, 3) for i in list_range(n)) + d_neg_6[j, l] - d_pos_6[j, l] == 4,
            f"Equation_27_{j}_{l}"
        )

$$
\sum_{i=1}^n \left( X_{ij4l} \right) * (4) + d_{7jl}^- - d_{7jl}^+ = 4, \; j = 1, 2, \ldots, m, \; l = 1, 2  \quad\quad \text{(28)}
$$

In [29]:
for j in list_range(m):
    for l in list_range(t):
        model.add_constraint(
            model.sum(X[i, j, 4, l] * person_competency(i, 4) for i in list_range(n)) + d_neg_7[j, l] - d_pos_7[j, l] == 4,
            f"Equation_28_{j}_{l}"
        )

$$
\sum_{i=1}^n \left( X_{ij5l} \right) * (5) + d_{8jl}^- - d_{8jl}^+ = 6, \; j = 1, 2, \ldots, m, \; l = 1, 2  \quad\quad \text{(29)}
$$

In [30]:
for j in list_range(m):
    for l in list_range(t):
        model.add_constraint(
            model.sum(X[i, j, 5, l] * person_competency(i, 5) for i in list_range(n)) + d_neg_8[j, l] - d_pos_8[j, l] == 6,
            f"Equation_29_{j}_{l}"
        )

$$
\sum_{i=1}^n \left( X_{ij6l} \right) * (6) + d_{9jl}^- - d_{9jl}^+ = 8, \; j = 1, 2, \ldots, m, \; l = 1, 2
 \quad\quad \text{(30)}
$$

In [31]:
for j in list_range(m):
    for l in list_range(t):
        model.add_constraint(
            model.sum(X[i, j, 6, l] * person_competency(i, 6) for i in list_range(n)) + d_neg_9[j, l] - d_pos_9[j, l] == 8,
            f"Equation_30_{j}_{l}"
        )

$$
\sum_{i=1}^n \left( X_{ij7l} \right) * (7) + d_{10jl}^- - d_{10jl}^+ = 5, \; j = 1, 2, \ldots, m, \; l = 1, 2  \quad\quad \text{(31)}
$$

In [32]:
for j in list_range(m):
    for l in list_range(t):
        model.add_constraint(
            model.sum(X[i, j, 7, l] * person_competency(i, 7) for i in list_range(n)) + d_neg_10[j, l] - d_pos_10[j, l] == 5,
            f"Equation_31_{j}_{l}"
        )

### Solve the problem
using CPLEX_PY CPLEX Version: 22.1.1.0

In [33]:
# Solve the model
solution = model.solve()

# Display the results
if solution:
    print("Objective Value:", solution.objective_value)

    # Collect solution values into a dictionary
    solution_dict = {var.name: var.solution_value for var in model.iter_variables()}
    save_solution_dict(solution_dict)

	# Create schedule based on solution value
    scheduler = Scheduler(solution=solution_dict)
    scheduler.get_schedule()

	# Prepare excel writer for visualization
    writer = Writer(
		file_name=scheduler.file_name, 
		num_personnel=scheduler.num_personnel,
		rows_per_person=scheduler.rows_per_person
	)

	# Write schedule in excel file
    writer.write(
		schedule=scheduler.schedule,
		day_shifts=scheduler.day_shifts,
		night_shifts=scheduler.night_shifts
	)

    # Convert the solution dictionary to a pandas DataFrame
    solution_df = pd.DataFrame(
        list(solution_dict.items()), columns=["Variable", "Value"]
    )

    # Save the DataFrame to an Excel file
    solution_df.to_excel("./src/solution/solution.xlsx", index=False)
    print("Solution saved to './src/solution/solution.xlsx'")
else:
    print("No solution found.")


Objective Value: 2618.9999999999986
Solution saved to './src/solution/solution.xlsx'


### Reference

[1] Kaçmaz, Ö., Alakaş, H. M., & Eren, T. (2019). Shift scheduling with the goal programming method: a case study in the glass industry. *Mathematics, 7*(6), 561. [https://doi.org/10.3390/math7060561](https://doi.org/10.3390/math7060561)
